<a href="https://colab.research.google.com/github/yyinsomnia/build-cngpt/blob/main/gpt_mao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 环境和依赖



In [1]:
# !wget https://raw.githubusercontent.com/yyinsomnia/openai-ex/main/input.txt

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-07-19 16:37:46--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-07-19 16:37:46 (26.6 MB/s) - ‘input.txt’ saved [1115394/1115394]



# 传统的语言模型bigram

In [2]:
with open('input.txt') as f:
    text = f.read()

In [3]:
print('length of the mao-poetic in characters is :', len(text))

length of the mao-poetic in characters is : 1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [6]:
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
stoi = {char:idx for idx, char in enumerate(chars)}
itos = {idx: char for idx, char in enumerate(chars)}

In [8]:
# print(stoi['北'])
# print(itos[273])

In [9]:
encode = lambda s: [stoi[c] for c in s]
decode = lambda ids: ''.join([itos[i] for i in ids])

In [10]:
# 毛爷爷的诗
# print(encode('北国风光'))
# print(decode([273, 415, 2025, 176]))

# shakespeare poetry
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [11]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [12]:
n = int(0.9 * len(data))
train_data = data[:n]
valid_data = data[n:]

In [13]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [14]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for i in range(block_size):
    context = x[:i+1]
    target = y[i]
    print(f"when context is {context}, target is {target}")

when context is tensor([18]), target is 47
when context is tensor([18, 47]), target is 56
when context is tensor([18, 47, 56]), target is 57
when context is tensor([18, 47, 56, 57]), target is 58
when context is tensor([18, 47, 56, 57, 58]), target is 1
when context is tensor([18, 47, 56, 57, 58,  1]), target is 15
when context is tensor([18, 47, 56, 57, 58,  1, 15]), target is 47
when context is tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is 58


In [15]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else valid_data
    ix = torch.randint(len(data) - block_size, (batch_size,))

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x, y

# print(get_batch('train'))
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [16]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when context is {context}, target is {target}')

when context is tensor([24]), target is 43
when context is tensor([24, 43]), target is 58
when context is tensor([24, 43, 58]), target is 5
when context is tensor([24, 43, 58,  5]), target is 57
when context is tensor([24, 43, 58,  5, 57]), target is 1
when context is tensor([24, 43, 58,  5, 57,  1]), target is 46
when context is tensor([24, 43, 58,  5, 57,  1, 46]), target is 43
when context is tensor([24, 43, 58,  5, 57,  1, 46, 43]), target is 39
when context is tensor([44]), target is 53
when context is tensor([44, 53]), target is 56
when context is tensor([44, 53, 56]), target is 1
when context is tensor([44, 53, 56,  1]), target is 58
when context is tensor([44, 53, 56,  1, 58]), target is 46
when context is tensor([44, 53, 56,  1, 58, 46]), target is 39
when context is tensor([44, 53, 56,  1, 58, 46, 39]), target is 58
when context is tensor([44, 53, 56,  1, 58, 46, 39, 58]), target is 1
when context is tensor([52]), target is 58
when context is tensor([52, 58]), target is 1
whe

In [17]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # 这里是有讲究的，这个embedding就是通过当前词预测下一个词，这是bigram吗？怎么感觉是unigram
        # 没问题，就是bigram。uniqgram就是每个词的出现只与它自己有关，bigram是每个词的出现与它的前一个有关
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

In [18]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [19]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [20]:
batch_size = 32
max_iter = 5000

for step in range(max_iter): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = m(xb, yb)

    if step % 100 == 0 or step == max_iter - 1:
        x_val, y_val = get_batch('valid')
        logits_val, loss_val = m(x_val, y_val)

        print(f'train loss: {loss}, valid loss: {loss_val}')


    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# 基本到2.2 就下不去了
print(loss.item())


train loss: 4.704006195068359, valid loss: 4.685050964355469
train loss: 4.681365013122559, valid loss: 4.604834079742432
train loss: 4.495052814483643, valid loss: 4.6050124168396
train loss: 4.401493549346924, valid loss: 4.348064422607422
train loss: 4.194651126861572, valid loss: 4.344674110412598
train loss: 4.22622013092041, valid loss: 4.2478928565979
train loss: 3.9928793907165527, valid loss: 4.014947891235352
train loss: 4.146249294281006, valid loss: 3.9515202045440674
train loss: 3.906283378601074, valid loss: 3.9018666744232178
train loss: 3.8001708984375, valid loss: 3.8397514820098877
train loss: 3.6482129096984863, valid loss: 3.764535665512085
train loss: 3.7147018909454346, valid loss: 3.621622085571289
train loss: 3.605959892272949, valid loss: 3.6587533950805664
train loss: 3.444122314453125, valid loss: 3.490373134613037
train loss: 3.327164888381958, valid loss: 3.3006374835968018
train loss: 3.322540044784546, valid loss: 3.474493980407715
train loss: 3.309330940

In [21]:
torch.zeros(1, 1, dtype=torch.long)

tensor([[0]])

In [22]:
# gen_idx = torch.tensor([encode('北国风光')], dtype=torch.long)
gen_idx = torch.zeros((1, 1), dtype=torch.long)
gen_idx

tensor([[0]])

In [23]:
print(decode(m.generate(idx = gen_idx, max_new_tokens=500)[0].tolist()))


were er:
BLenEr w
Se, WWhes jFYNapay tethore!uE:
GLBedofubl'VOKuit!
Sit LINCL:
IZKII R.
tr han:
Theaseise athicof
LOLBus CERYde tDesula tttithe lour wes kisasiseaUTw?Oinounera tlantUFKGave
IBFr nein t;AUBo p$s
I& 'switoud su, r t ide
xxatqe f boo te tt, ho morved'WAce t f Vn?k:
KIOWoid,
VORKoy CHAneyed ng princr whe, dme yFoouthyVOy
ARGongas:
TIDUEn t$YCVOUCKIOPaus s tathainsullls serd!
A:
ILINord chatawoureY VBul CHakerave t ty SKINGAig?O ad omapy d?LOThof thot.I:
Ansinde ow.
S:

Gloldin a I'Q?


# attention的数学技巧

In [24]:
torch.manual_seed(42)
# 下三角矩阵就能做到和左边的词元attention
a = torch.tril(torch.ones(3, 3))
# 但是这里我们要模拟的不是mask，而是一个K * Q得到的attention权重矩阵，还得归一化
a = a / torch.sum(a, 1, keepdim=True)

b = torch.randint(0,10,(3,2)).float()

c = a @ b
print('a=')
print(a)
print('---')
print('b=')
print(b)
print('c=')
print('---')
print(c)

# c look as the column combination of a
# 2.  0. 0  2
# 1.  3. 0. 4
# 2/3 2  2. 4.6666

# 但是这里应该看成 row combination of b
# 1.0 * [2., 7.] + 0.0 * [6., 4.] + 0.0 * [6., 5.] 这就是第1个token的BOW表示
# 0.5 * [2., 7.] + 0.5 * [6., 4.] + 0.0 * [6., 5.] 这就是第2个token的BOW表示
# 0.3 * [2., 7.] + 0.3 * [6., 4.] + 0.3 * [6., 5.] 这就是第3个token的BOW表示


a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
---
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [25]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
print(x)

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])


In [26]:
# 先用朴素BOW来表达？
# x[b, t] = mean_{i <= t} x[b, i]

xbow = torch.zeros(B, T, C)
for b in range(B):
    for t in range(T):
        # 私以为比Andrej老师的变量名要好，因为这里也包含了step t这个token
        x_t_tprev = x[b, :t+1] # shape (t, C)
        xbow[b, t] = torch.mean(x_t_tprev, 0)

print(xbow)

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])


In [27]:
# 再用上面那个tril的矩阵乘法来表示
# 为什么用tril？因为不用写for循环，并行度更高

wei = torch.tril(torch.ones(T, T))
print('wei_tril:')
print(wei)


# 这里keepdim=True shape是(8, 1)所以除法就是每行除以对应的那一行的1个sum出来的数
# keepdim=False, shape是(8, ), 那就是每一行从左到右每个元素都找到对应的去除
wei_sum = wei.sum(1, keepdim=True)

print('wei_sum:')
print(wei_sum)
wei = wei / wei_sum


print('wei:')
print(wei)

xbow2 = wei @ x

# 发现太小的值，这里[1,5,1] 是 0.0020，会导致allclose判断为False，
# 这个问题很有意思，我记得21年跑这个代码没有这个问题，现在出现了。我也运行了一下Andrej的原始notebook也是Fasle
# 说明大概率是python和torch的版本升级导致的，可能是seed rand的数字不一样了
print('xbow2:')
print(xbow2[1][5][1])
print(xbow2[1][5][1] - xbow[1][5][1])
torch.allclose(xbow[1][5][1], xbow2[1][5][1])

wei_tril:
tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
wei_sum:
tensor([[1.],
        [2.],
        [3.],
        [4.],
        [5.],
        [6.],
        [7.],
        [8.]])
wei:
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        

False

In [28]:
# 用softmax
# 为什么要用softmax？后续如果是模型过完一些层，得出来的是logits而不是probs
# logits范围(-inf, inf)，而不是像上面都是类似probs的 1/n 范围[0.0, 1.0]
# 过一下softmax，范围就到[0.0, 1.0]了

wei = torch.tril(torch.ones(T, T))
wei = torch.where(wei == 0.0, float('-inf'), wei)

"""
在PyTorch中，`F.softmax` 函数用于沿着指定的维度对输入张量进行softmax操作。对于张量 `wei`，其形状为 `(T, T)`，让我们分析一下 `F.softmax(wei, 1)` 和 `F.softmax(wei, -1)` 的含义：

1. **`F.softmax(wei, 1)`**:
   - 这里的 `1` 表示在第1个维度（即列方向）上进行softmax操作。对于每一行，`F.softmax` 会计算该行内的softmax值。

2. **`F.softmax(wei, -1)`**:
   - 这里的 `-1` 表示在最后一个维度上进行softmax操作。对于二维张量 `(T, T)` 来说，最后一个维度也是列方向（即维度1）。因此，`F.softmax(wei, -1)` 实际上与 `F.softmax(wei, 1)` 是等价的。

总结来说，`F.softmax(wei, 1)` 和 `F.softmax(wei, -1)` 在形状为 `(T, T)` 的张量上执行时，是相同的操作，因为它们都在列方向上计算softmax。
"""
wei = F.softmax(wei, 1)
print('wei:')
print(wei)

xbow3 = wei @ x

# xbow2和xbow3是一样的
# 因为xbow2和xbow3都是先除再加，xbow的mean是先加再除
torch.allclose(xbow2, xbow3)

wei:
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


True

In [29]:
# 终于来了！自注意力self-attention
torch.manual_seed(1337)

B, T, C = 2, 16, 8

x = torch.randn(B, T, C)

header_size = 4

key = nn.Linear(C, header_size)
query = nn.Linear(C, header_size)
value = nn.Linear(C, header_size)

# k,q,v shape (B, T, head_size)
k = key(x)
q = query(x)
v = value(x)

tril = torch.tril(torch.ones(T, T))
wei = q @ k.transpose(1, 2) # shape (B, T, T)

# 这里wei和tril的shape不一样，看来是能自动boardcast的
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, -1)

x_attent_left = wei @ v

x_attent_left.shape # shape (B, T, header_size)

torch.Size([2, 16, 4])

注意：
- 注意力是一种**交流机制**。可以看成是有向图中的节点互相关注，并将所有指向它们的节点的信息加权汇总，权重取决于数据。
- 没有空间概念。注意力只是作用于一组向量。这就是我们需要对标记进行位置编码的原因。
- 在 "编码器 "注意力区块中，只需删除用下三角矩阵进行屏蔽的一行，就可以让所有标记进行交流。这里的区块被称为 "解码器 "注意区块，因为它具有下三角形掩码，通常用于自回归设置，如语言建模。
- "自注意力"只是指键和值与查询的来源相同。在 "交叉注意力"中，查询仍由 x 生成，但键和值来自其他外部来源（如编码器模块）。
- "缩放 "注意力额外将 wei 除以 1/sqrt（head_size）。这样，当输入 Q、K 为单位方差时，wei 也将是单位方差，Softmax 将保持分散，不会过于饱和。说明如下

In [30]:
q = torch.randn(8, 4) # shape (T, head_size)
print(f'q var: {q.var()}')

k = torch.randn(8, 4) # shape (T, head_size)
print(f'k var: {k.var()}')

wei = q @ k.transpose(0, 1) # shape (T, T)
print(f'wei var: {wei.var()}')

q var: 0.55227130651474
k var: 1.0105643272399902
wei var: 2.630138874053955


In [31]:
wei_norm = wei / (4 ** 1/2) # head_size is 4

print(f'wei_norm vars: {wei_norm.var()}')

wei_norm vars: 0.6575347185134888


In [32]:
# 方差大了过完softmax会变得过于峰值(peak)

# 设置打印选项，不使用科学计数法
torch.set_printoptions(sci_mode=False)

wei_probs = F.softmax(wei, dim=1)
print(f'wei_probs: {wei_probs}') # 可以看到[3, 3] 不用科学计数法都是0.0000了，这样整个向量会趋向于one-hot
# one-hot有什么不好呢？个人的想法是这里wei是每个token和其他token的关系权重，两两之间总归是有一些关系的

wei_norm_probs = F.softmax(wei_norm, dim=1)
print(f'wei_norm_probs: {wei_norm_probs}')

wei_probs: tensor([[    0.0127,     0.1197,     0.0321,     0.2054,     0.0137,     0.5546,
             0.0208,     0.0410],
        [    0.0285,     0.0135,     0.0004,     0.1478,     0.0122,     0.0009,
             0.1557,     0.6410],
        [    0.0442,     0.0271,     0.0717,     0.0142,     0.6493,     0.0171,
             0.1586,     0.0178],
        [    0.0960,     0.0813,     0.5371,     0.0106,     0.0814,     0.1663,
             0.0118,     0.0155],
        [    0.0609,     0.0431,     0.0761,     0.1198,     0.1977,     0.0493,
             0.4034,     0.0498],
        [    0.3244,     0.0644,     0.2650,     0.0174,     0.1623,     0.0263,
             0.0584,     0.0818],
        [    0.0924,     0.0186,     0.0485,     0.0088,     0.6168,     0.0040,
             0.1757,     0.0353],
        [    0.3909,     0.0708,     0.1346,     0.0087,     0.2104,     0.0115,
             0.0322,     0.1409]])
wei_norm_probs: tensor([[0.0491, 0.1505, 0.0779, 0.1971, 0.0508, 0.3

# 完整的实现GPT

In [38]:
class GPTConfig:
    def __init__(self, config):
        self.B = config['B']
        self.T = config['T']
        self.C = config['C']

        self.vocab_size = config['vocab_size']

        self.header_size = config['header_size']
        self.headers_num = config['headers_num'] # 要不先不实现多头，跑毛爷爷的诗，单头可能也可以了

        self.blocks_num = config['blocks_num']
        self.ffn_size = config['ffn_size']

class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.key = nn.Linear(config.C, config.header_size * config.headers_num)
        self.query = nn.Linear(config.C, config.header_size * config.headers_num)
        self.value = nn.Linear(config.C, config.header_size * config.headers_num)
        self.attn_fc = nn.Linear(config.header_size * config.headers_num, config.C)
        # self.feedforward = nn.Linear(config.header, config.ffn_size)
        # self.feedforward = nn.Linear(config.header, config.C) # ffn_size应该就是C，不然过下一个block的时候dim和过完embd的不一样了
        # 这样应该能学习更多的参数，也不会导致过下一个block的dim不一样
        # 但是我没明白的是直接这么提升维度，信息从何而来呢？这就有点像是线性代数里面平行的vector
        self.ff1 = nn.Linear(config.C, config.ffn_size)
        self.ff2 = nn.Linear(config.ffn_size, config.C)

    def split_headers(self, x, batch_size):
        """Split the last dimension into (num_heads, head_dim) and transpose the result."""
        x = x.view(batch_size, config.T, self.config.headers_num, self.config.header_size)

        return x.transpose(1, 2) # shape (B, headers_num, T, header_size) 这里需要thinking一下why

    def combine_headers(self, x, batch_size):
        """
            Combine the heads and restore the original last dimension.
            x.shape (B, headers_num, T, header_size)
        """
        x = x.transpose(1, 2).contiguous()
        # 这里view中的-1，表示自动计算。其实可以直接用T
        return x.view(batch_size, self.config.T, self.config.headers_num * self.config.header_size)

    def attn(self, x):
        """
        x.shape (B, T, C)
        """
        config = self.config
        B, T, C = x.shape

        # k, q, v shape (B, headers_num, T, header_size)
        k = self.split_headers(self.key(x), B)
        q = self.split_headers(self.query(x), B)
        v = self.split_headers(self.value(x), B)

        tril = torch.tril(torch.ones(config.T, config.T))
        wei = q @ k.transpose(-2, -1) # 这个时候用负数的兼容性就体现了
        wei = wei.masked_fill(tril == 0, float('-inf'))
        wei = wei / (config.header_size ** 0.5)
        wei = F.softmax(wei, dim=-1)
        # wei.shape (B, headers_num, T, T)

        out = wei @ v # out.shape (B, headers_num, T, header_size)
        out = self.combine_headers(out, B)  # out.shape(B, T, config.header_size * config.headers_num)
        x = self.attn_fc(out)

        return x

    def ffn(self, x):
        x = self.ff1(x)
        x = F.gelu(x, approximate='tanh')
        out2 = self.ff2(x)
        return out2

    def layernorm(self, x, epsilon=1e-5):
        mean = torch.mean(x, axis=-1, keepdims=True)
        std = torch.sqrt(x.var(axis=-1, keepdims=True) + epsilon)
        norm_x = (x - mean) / std
        return norm_x

    def forward(self, x):
        # 应该还有一个norm，为什么要norm有点忘了，先不实现
        x = x + self.attn(self.layernorm(x))
        x = x + self.ffn(self.layernorm(x))
        return x



class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.wte = nn.Embedding(config.vocab_size, config.C)
        self.wpe = nn.Embedding(config.T, config.C)
        self.blocks = nn.ModuleList([Block(config) for _ in range(config.blocks_num)])
        # why lm_fc. 你可以🤔，每个position都是aggregate左侧的信息，然后再深度计算
        # 最后你的task是next token prediction
        # 所以你要把这个"aggregate左侧的信息，然后再深度计算"的信息，映射到token embd空间
        # 然后通过wte逆向 映射出来 tokenid
        self.lm_fc = nn.Linear(config.C, config.C)


    def forward(self, idx, targets=None):
        B, T = idx.shape
        token_idx = idx
        range_tensor = torch.arange(T)
        position_idx = range_tensor.expand(B, T)

        x = self.wte(token_idx) + self.wpe(position_idx)
        for block in self.blocks:
            x = block(x)
        x = self.lm_fc(x) # (B, T, C)
        # 使用嵌入层的权重转置进行反向映射
        logits = torch.matmul(x, self.wte.weight.t())


        if targets is None:
            loss = None
        else:
            targets = targets.view(B*T)
             # 要softmax吗？没想明白
            logits = logits.view(B*T, vocab_size)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens=50):

        for step in range(max_new_tokens):
            _B, seq_len = idx.shape
            if seq_len > self.config.T:
                idx_fixedlen = idx[:, -self.config.T:]
                _B, seq_len = idx_fixedlen.shape
            else:

                padding_len = self.config.T - seq_len
                idx_fixedlen = F.pad(idx, (0, padding_len), "constant", 0)

            logits, loss = self(idx_fixedlen)
            logits = logits[:, seq_len - 1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)

        return idx


config = GPTConfig({'B': 16, 'T': 32, 'C': 64, 'vocab_size': vocab_size, 'header_size':16, 'headers_num': 4, 'blocks_num': 4, 'ffn_size': 256})
m = GPT(config)

# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# idx = encode('北国风光')
# print(idx)

batch_size = config.B
block_size = config.T

def get_batch(split):
    data = train_data if split == 'train' else valid_data
    ix = torch.randint(len(data) - block_size, (batch_size,))

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')

print('token id 0:' + decode([0]))

print(f'xb: {xb.shape}')
print(f'vocab_size: {vocab_size}')

print(f'wpe: {m.wpe}')

logits, loss = m(xb, yb)
print(f'logits: {logits}')
print(f'loss: {loss}')

# gen_idx = torch.tensor([encode('北国风光')], dtype=torch.long)
gen_idx = torch.zeros((1, 1), dtype=torch.long)
print(f'gen_idx: {gen_idx.shape}')
print(decode(m.generate(idx = gen_idx, max_new_tokens=500)[0].tolist()))

0.20928 M parameters
token id 0:

xb: torch.Size([16, 32])
vocab_size: 65
wpe: Embedding(32, 64)
logits: tensor([[ -8.7881, -12.6831,  15.4393,  ...,   5.0796,  -2.8202,  10.1654],
        [  0.1913,  -1.2256,   2.1896,  ...,   2.7092,   8.2599,  -8.5259],
        [ -1.8433,  -1.7806,  -6.3485,  ...,   2.7871,  -7.8156,  -6.5514],
        ...,
        [  0.8148,  -3.6508,  -2.7441,  ...,  -1.7563,  -3.1374,  -3.5158],
        [  1.6214, -12.0208,   9.7542,  ...,  -6.4837,  -1.8898,  12.8367],
        [  2.4415,  -2.6835,  -5.8897,  ...,  -2.1421,  -2.8250,   5.7181]],
       grad_fn=<ViewBackward0>)
loss: 16.77750587463379
gen_idx: torch.Size([1, 1])

Vy$F3?bd3OTh.nCQnrLKh.T?3QWWdCz333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333o$3333333333333333333333333333333333333333333333333333333333333333333333333333

In [39]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [40]:
eval_interval = 100
max_iter = 5000

for iter in range(max_iter):
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)

    if iter % eval_interval == 0 or iter == max_iter:
        x_val, y_val = get_batch('valid')
        logits_val, loss_val = m(x_val, y_val)
        print(f'step: {iter}, train loss: {loss}, valid loss:{loss_val}')

    optimizer.zero_grad(set_to_none=True)

    loss.backward()

    optimizer.step()

# 跑了半天loss 3.x，比bigram的还高。多跑几轮能到2.5，loss有波动
# 增加header_size，blocks_num， batch_size 没啥用
# 还有几个没有优化的:1. 非线性激活 2.LayerNorm 3. MultiHead
# 看了下Andrej老师的notebook，step5000 trainloss能降到1.6,valloss能降到1.8
# 这里可能有2个原因，1是数据集毛爷爷的诗还是不够多，然后vocab比英语的chars要大不少；2是代码有bug，
# 但是还是要想明白一点，预训练和SFT可能不太一样，loss不可能逼近0，因为譬如第一个字去预测第二个字，总是会有loss的，加大block_size，平均下来应该能降低一些loss

step: 0, train loss: 16.2195987701416, valid loss:16.739917755126953
step: 100, train loss: 2.8510680198669434, valid loss:2.930788040161133
step: 200, train loss: 2.7266013622283936, valid loss:2.6504437923431396
step: 300, train loss: 2.5705726146698, valid loss:2.63372802734375
step: 400, train loss: 2.5987977981567383, valid loss:2.5321881771087646
step: 500, train loss: 2.448687791824341, valid loss:2.4691874980926514
step: 600, train loss: 2.259223222732544, valid loss:2.383213520050049
step: 700, train loss: 2.347440004348755, valid loss:2.362677574157715
step: 800, train loss: 2.223034381866455, valid loss:2.3258039951324463
step: 900, train loss: 2.2427568435668945, valid loss:2.2008426189422607
step: 1000, train loss: 2.170031785964966, valid loss:2.1725106239318848
step: 1100, train loss: 2.1783382892608643, valid loss:2.079711675643921
step: 1200, train loss: 2.0962235927581787, valid loss:2.289428234100342
step: 1300, train loss: 2.0045337677001953, valid loss:2.1986746788

In [36]:
gen_idx = torch.tensor([encode('北国风光')], dtype=torch.long)
print(decode(m.generate(idx = gen_idx, max_new_tokens=500)[0].tolist())) # 实际效果也一般

KeyError: '北'